# Riemannian Geometry for Molecular Surface Approximation (RGMolSA)

**Contributing Authors:** Stuart J. Hall, Daniel J. Cole, Thomas Murphy, Rachael M. E. Pirie

This notebook accompanies our [paper](https://arxiv.org/pdf/2301.04424.pdf) detailing a novel method for molecular surface approximation based on Riemannian Geometry. The descriptor produced in this notebook represents the (2k+1)x(2k+1) Hermitian matrix associated with the surface. 

This notebook can be used to replicate the small screen detailed in the paper, comparing the shape of a series of PDE5 inhibitors known to have similar shape. Note that the method is currently still under development, and this notebook is not designed to be used for full virtual screens. 

In [ ]:
import sys
import os
sys.path.append('../KQMolSA/scripts')  # replace with path to python scripts
import numpy as np

from utils import get_score
from get_descriptor import get_descriptor

from rdkit.Chem import PandasTools, Draw

root = '../KQMolSA/data'  # specify directory containing data

In [ ]:
# define values of k to consider
k_vals = [1, 2]

## Single Conformer

The following cells compute the descriptor and calculate the similarity of a single conformer for each of Sildenafil, Vardenafil and Tadalafil. 

In [ ]:
# load data
data = PandasTools.LoadSDF(os.path.join(root, 'SVT.sdf'))  # replace with path to data
mols = list(data['ROMol'])

In [ ]:
# visualise molecules
Draw.MolsToGridImage(mols)

In [ ]:
# get the shape descriptor for each molecule in the list
descriptor = [get_descriptor(mol, k_vals) for mol in mols]

print(descriptor)

In [ ]:
# compute the distance between molecules
# function returns the absolute distance and the similarity score between 0 and 1 and "self" for self comparison

scores1, scores2 = np.empty((len(descriptor), len(descriptor)), dtype=object), np.empty((len(descriptor), len(descriptor)), dtype=object)

for i in range(len(descriptor)):
    sa_query = descriptor[i].surface_area
    for j in range(len(descriptor)):
        sa_test = descriptor[j].surface_area
        d_k1, sim_score_k1, x0 = get_score(descriptor[i].kq_shape[0], descriptor[j].kq_shape[0], sa_query, sa_test, k_vals[0], i, j)
        d_k2, sim_score_k2, x0 = get_score(descriptor[i].kq_shape[1], descriptor[j].kq_shape[1], sa_query, sa_test, k_vals[1], i, j, x0)
        
        #append shape scores for each k
        scores1[i][j] = sim_score_k1
        scores2[i][j] = sim_score_k2

In [ ]:
print(scores1)

In [ ]:
print(scores2)

## Multi-Conformer Similarity

The following cells compute the similarity between different conformers of the same molecule. The supplied datasets provide 11 random and 11 low energy conformers for Sildenafil, Vardenafil and Tadalafil, generated using RDKit. 

In [ ]:
# Sildenafil Random Conformers

# load data
data = PandasTools.LoadSDF(os.path.join(root, 'sildenafil_confs_10random.sdf')) 
sildenafil_random = list(data['ROMol'])

# get descriptors
sildenafil_random_descriptors = [get_descriptor(mol, k_vals) for mol in sildenafil_random]  

# compute similarity
scores_sildenafil_random1, scores_sildenafil_random2 = np.empty((len(sildenafil_random_descriptors), len(sildenafil_random_descriptors)), dtype=object), np.empty((len(sildenafil_random_descriptors), len(sildenafil_random_descriptors)), dtype=object)

for i in range(len(sildenafil_random_descriptors)):
    sa_query = sildenafil_random_descriptors[i].surface_area
    for j in range(len(sildenafil_random_descriptors)):
        sa_test = sildenafil_random_descriptors[j].surface_area
        d_k1, sim_score_k1, x0 = get_score(sildenafil_random_descriptors[i].kq_shape[0], sildenafil_random_descriptors[j].kq_shape[0], sa_query, sa_test, k_vals[0], i, j)
        d_k2, sim_score_k2, x0 = get_score(sildenafil_random_descriptors[i].kq_shape[1], sildenafil_random_descriptors[j].kq_shape[1], sa_query, sa_test, k_vals[1], i, j, x0)
        
        #append shape scores for each k
        scores_sildenafil_random1[i][j] = sim_score_k1
        scores_sildenafil_random2[i][j] = sim_score_k2

print(scores_sildenafil_random1)
print(scores_sildenafil_random2)

In [ ]:
# Sildenafil Low Energy Conformers

# load data
data = PandasTools.LoadSDF(os.path.join(root, 'sildenafil_confs_10.sdf')) 
sildenafil_lowe = list(data['ROMol'])

# get descriptors
sildenafil_lowe_descriptors = [get_descriptor(mol, k_vals) for mol in sildenafil_lowe] 

# compute similarity
scores_sildenafil_lowe1, scores_sildenafil_lowe2 = np.empty((len(sildenafil_lowe_descriptors), len(sildenafil_lowe_descriptors)), dtype=object), np.empty((len(sildenafil_lowe_descriptors), len(sildenafil_lowe_descriptors)), dtype=object)

for i in range(len(sildenafil_lowe_descriptors)):
    sa_query = sildenafil_lowe_descriptors[i].surface_area
    for j in range(len(sildenafil_lowe_descriptors)):
        sa_test = sildenafil_lowe_descriptors[j].surface_area
        d_k1, sim_score_k1, x0 = get_score(sildenafil_lowe_descriptors[i].kq_shape[0], sildenafil_lowe_descriptors[j].kq_shape[0], sa_query, sa_test, k_vals[0], i, j)
        d_k2, sim_score_k2, x0 = get_score(sildenafil_lowe_descriptors[i].kq_shape[1], sildenafil_lowe_descriptors[j].kq_shape[1], sa_query, sa_test, k_vals[1], i, j, x0)
        
        #append shape scores for each k
        scores_sildenafil_lowe1[i][j] = sim_score_k1
        scores_sildenafil_lowe2[i][j] = sim_score_k2

print('Scores k=1: ')        
print(scores_sildenafil_lowe1)
print('Scores k=2: ') 
print(scores_sildenafil_lowe2)

In [ ]:
# Vardenafil Random Conformers

# load data
data = PandasTools.LoadSDF(os.path.join(root, 'vardenafil_confs_10random.sdf')) 
vardenafil_random = list(data['ROMol'])

# get descriptors
vardenafil_random_descriptors = [get_descriptor(mol, k_vals) for mol in vardenafil_random]  

# compute similarity
scores_vardenafil_random1, scores_vardenafil_random2 = np.empty((len(vardenafil_random_descriptors), len(vardenafil_random_descriptors)), dtype=object), np.empty((len(vardenafil_random_descriptors), len(vardenafil_random_descriptors)), dtype=object)

for i in range(len(vardenafil_random_descriptors)):
    sa_query = vardenafil_random_descriptors[i].surface_area
    for j in range(len(vardenafil_random_descriptors)):
        sa_test = vardenafil_random_descriptors[j].surface_area
        d_k1, sim_score_k1, x0 = get_score(vardenafil_random_descriptors[i].kq_shape[0], vardenafil_random_descriptors[j].kq_shape[0], sa_query, sa_test, k_vals[0], i, j)
        d_k2, sim_score_k2, x0 = get_score(vardenafil_random_descriptors[i].kq_shape[1], vardenafil_random_descriptors[j].kq_shape[1], sa_query, sa_test, k_vals[1], i, j, x0)
        
        #append shape scores for each k
        scores_vardenafil_random1[i][j] = sim_score_k1
        scores_vardenafil_random2[i][j] = sim_score_k2

print('Scores k=1: ')         
print(scores_vardenafil_random1)
print('Scores k=2: ') 
print(scores_vardenafil_random2)

In [ ]:
# Vardenafil Low Energy Conformers

# load data
data = PandasTools.LoadSDF(os.path.join(root, 'vardenafil_confs_10.sdf')) 
vardenafil_lowe = list(data['ROMol'])

# get descriptors
vardenafil_lowe_descriptors = [get_descriptor(mol, k_vals) for mol in vardenafil_lowe] 

# compute similarity
scores_vardenafil_lowe1, scores_vardenafil_lowe2 = np.empty((len(vardenafil_lowe_descriptors), len(vardenafil_lowe_descriptors)), dtype=object), np.empty((len(vardenafil_lowe_descriptors), len(vardenafil_lowe_descriptors)), dtype=object)

for i in range(len(vardenafil_lowe_descriptors)):
    sa_query = vardenafil_lowe_descriptors[i].surface_area
    for j in range(len(vardenafil_lowe_descriptors)):
        sa_test = vardenafil_lowe_descriptors[j].surface_area
        d_k1, sim_score_k1, x0 = get_score(vardenafil_lowe_descriptors[i].kq_shape[0], vardenafil_lowe_descriptors[j].kq_shape[0], sa_query, sa_test, k_vals[0], i, j)
        d_k2, sim_score_k2, x0 = get_score(vardenafil_lowe_descriptors[i].kq_shape[1], vardenafil_lowe_descriptors[j].kq_shape[1], sa_query, sa_test, k_vals[1], i, j, x0)
        
        #append shape scores for each k
        scores_vardenafil_lowe1[i][j] = sim_score_k1
        scores_vardenafil_lowe2[i][j] = sim_score_k2

print('Scores k=1: ') 
print(scores_vardenafil_lowe1)
print('Scores k=2: ') 
print(scores_vardenafil_lowe2)

In [ ]:
# Tadalafil Random Conformers

# load data
data = PandasTools.LoadSDF(os.path.join(root, 'tadalafil_confs_10random.sdf')) 
tadalafil_random = list(data['ROMol'])

# get descriptors
tadalafil_random_descriptors = [get_descriptor(mol, k_vals) for mol in tadalafil_random]  

# compute similarity
scores_tadalafil_random1, scores_tadalafil_random2 = np.empty((len(tadalafil_random_descriptors), len(tadalafil_random_descriptors)), dtype=object), np.empty((len(tadalafil_random_descriptors), len(tadalafil_random_descriptors)), dtype=object)

for i in range(len(tadalafil_random_descriptors)):
    sa_query = tadalafil_random_descriptors[i].surface_area
    for j in range(len(tadalafil_random_descriptors)):
        sa_test = tadalafil_random_descriptors[j].surface_area
        d_k1, sim_score_k1, x0 = get_score(tadalafil_random_descriptors[i].kq_shape[0], tadalafil_random_descriptors[j].kq_shape[0], sa_query, sa_test, k_vals[0], i, j)
        d_k2, sim_score_k2, x0 = get_score(tadalafil_random_descriptors[i].kq_shape[1], tadalafil_random_descriptors[j].kq_shape[1], sa_query, sa_test, k_vals[1], i, j, x0)
        
        #append shape scores for each k
        scores_tadalafil_random1[i][j] = sim_score_k1
        scores_tadalafil_random2[i][j] = sim_score_k2

print('Scores k=1: ')         
print(scores_tadalafil_random1)
print('Scores k=2: ') 
print(scores_tadalafil_random2)

In [ ]:
# Tadalafil Low Energy Conformers

# load data
data = PandasTools.LoadSDF(os.path.join(root, 'tadalafil_confs_10.sdf')) 
tadalafil_lowe = list(data['ROMol'])

# get descriptors
tadalafil_lowe_descriptors = [get_descriptor(mol, k_vals) for mol in tadalafil_lowe] 

# compute similarity
scores_tadalafil_lowe1, scores_tadalafil_lowe2 = np.empty((len(tadalafil_lowe_descriptors), len(tadalafil_lowe_descriptors)), dtype=object), np.empty((len(tadalafil_lowe_descriptors), len(tadalafil_lowe_descriptors)), dtype=object)

for i in range(len(tadalafil_lowe_descriptors)):
    sa_query = tadalafil_lowe_descriptors[i].surface_area
    for j in range(len(tadalafil_lowe_descriptors)):
        sa_test = tadalafil_lowe_descriptors[j].surface_area
        d_k1, sim_score_k1, x0 = get_score(tadalafil_lowe_descriptors[i].kq_shape[0], tadalafil_lowe_descriptors[j].kq_shape[0], sa_query, sa_test, k_vals[0], i, j)
        d_k2, sim_score_k2, x0 = get_score(tadalafil_lowe_descriptors[i].kq_shape[1], tadalafil_lowe_descriptors[j].kq_shape[1], sa_query, sa_test, k_vals[1], i, j, x0)
        
        #append shape scores for each k
        scores_tadalafil_lowe1[i][j] = sim_score_k1
        scores_tadalafil_lowe2[i][j] = sim_score_k2

print('Scores k=1: ') 
print(scores_tadalafil_lowe1)
print('Scores k=2: ') 
print(scores_tadalafil_lowe2)